In [ ]:

import numpy as np
import segmentation.reggrow.build.lib.reggrow as rg
pcd = rg.PointCloud.from_ply("./data/demo.ply", True)
isinstance_inds = np.ones(pcd.size, dtype=np.int32) * -1
rg.region_growing(
    pcd,
    isinstance_inds,
    1,1,1,
    verbose=False
)

Loading ./data/demo.ply
Read 942188 points
Info: Generated by CloudCompare!
Fields: vertex: x, y, z, red, green, blue, scalar_Original_cloud_index, 
DEBUG: init parameters
Building KDTree
DEBUG: subdivvide into chunks
DEBUG: creating threads loop
Chunk_id: 0	Num. Points: 942188
DEBUG: computing normals
Starting cleanup.


In [6]:
from annotation import IndexSegmentation

reggrow_idx=0
pcd = rg.PointCloud.from_ply("./data/demo.ply", True)
reggrow_segmentation = IndexSegmentation(f'region_growing_{reggrow_idx:03}', pcd.size)
rg.region_growing(
    pcd,
    reggrow_segmentation.instance_idx,
    1,1,1,
    verbose=False
)


Loading ./data/demo.ply
Read 942188 points
Info: Generated by CloudCompare!
Fields: vertex: x, y, z, red, green, blue, scalar_Original_cloud_index, 
DEBUG: init parameters
Building KDTree
DEBUG: subdivvide into chunks
DEBUG: creating threads loop
Chunk_id: 0	Num. Points: 942188
DEBUG: computing normals
Starting cleanup.
